# Visualizing the nuScenes Dataset


This notebook introduces the nuScenes dataset and visualizes the information relevant for the trajectory prediction task.  
We explore:

- Dataset structure (JSON tables)
- Sensor data (camera + lidar)
- HD maps (drivable area)
- Past and future trajectories of agents
- Rasterized input representation used by the MTP model
- Agent State Vector (velocity, acceleration, heading-rate)

nuScenes is a large multimodal dataset, but for prediction we only use a very specific subset of information.


In [ ]:
import sys
import os
from pathlib import Path


current_path = Path(os.getcwd())
project_root = current_path.parent

if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    import nuimages 
    import nuscenes
    print("nuimages and nuscenes imported")
except ImportError as e:
    print(f"error: {e}")

First we import all the necessary libraries

In [ ]:
import argparse
import pickle
from nuscenes import NuScenes
import os
import json
from nuscenes.map_expansion.map_api import NuScenesMap

from pyquaternion import Quaternion
from nuscenes.utils.geometry_utils import transform_matrix
from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.compute_metrics import compute_metrics
import torch
import numpy as np
from pyquaternion import Quaternion
from nuscenes.eval.prediction.data_classes import Prediction
from nuscenes.eval.prediction.compute_metrics import compute_metrics
from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle
from nuscenes.eval.prediction.data_classes import Prediction

from nuscenes import NuScenes
from nuscenes.prediction import PredictHelper
from nuscenes.eval.prediction.data_classes import Prediction
from nuscenes.eval.prediction.metrics import MinADEK, MinFDEK, RowMean
from nuscenes.prediction.models.backbone import ResNetBackbone

from matplotlib import pyplot as plt

from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.compute_metrics import compute_metrics
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.input_representation.static_layers import StaticLayerRasterizer
from nuscenes.prediction.input_representation.agents import AgentBoxesWithFadedHistory
from nuscenes.prediction.input_representation.interface import InputRepresentation
from nuscenes.prediction.input_representation.combinators import Rasterizer

from nuscenes.eval.prediction.config import load_prediction_config
from nuscenes.eval.prediction.splits import get_prediction_challenge_split
from nuscenes.prediction import PredictHelper
from nuscenes.prediction.models.physics import ConstantVelocityHeading, PhysicsOracle

from nuscenes.map_expansion import arcline_path_utils
from nuscenes.prediction.models.backbone import ResNetBackbone
from nuscenes.prediction.models.mtp import MTP
from nuscenes.prediction.models.covernet import CoverNet

from torch.utils.data import Dataset, DataLoader
from nuscenes.prediction.models.mtp import MTPLoss

In [ ]:
DATAROOT = '/root/nuscenes-devkit/data/sets/nuscenes'

nuscenes = NuScenes('v1.0-mini', dataroot=DATAROOT)

Summary of dataset size

In [ ]:
print("Number of scenes:", len(nuscenes.scene))
print("Number of annotations:", len(nuscenes.sample_annotation))
print("Number of keyframe samples:", len(nuscenes.sample))
print("Number of tracked instances:", len(nuscenes.instance))

List all JSON metadata files in v1.0-mini

In [ ]:
version_dir = os.path.join(DATAROOT, "v1.0-mini")
json_files = [f for f in os.listdir(version_dir) if f.endswith(".json")]

print("JSON files available in v1.0-mini:\n")
for f in json_files:
    print(" -", f)


Preview of the structure of the key JSON tables

In [ ]:
def show_json_keys(filename, n=5):
    print(f"\nInside file: {filename}:")
    path = os.path.join(version_dir, filename)
    data = json.load(open(path))
    for key in data[0]:
        print(" •", key)
    print(f"Example:\n{data[0]}")
    
show_json_keys("sample.json")
show_json_keys("sample_annotation.json")
show_json_keys("instance.json")
show_json_keys("ego_pose.json")


Visuallization of sensors of a scene

In [ ]:
nuscenes.render_sample_data(nuscenes.sample[0]['data']['CAM_FRONT'])
nuscenes.render_sample_data(nuscenes.sample[0]['data']['LIDAR_TOP'])
nuscenes.render_sample_data(nuscenes.sample[0]['data']['RADAR_FRONT'])

Visualization of drivable area map

In [ ]:
sample_token = nuscenes.sample[0]['token']
log = nuscenes.get('log', nuscenes.get('scene', nuscenes.sample[0]['scene_token'])['log_token'])
map_name = log['location']
nmap = NuScenesMap(dataroot=DATAROOT, map_name=map_name)

nmap.render_map_in_image(nusc=nuscenes, sample_token=sample_token, layer_names=['road_segment', 'lane', 'ped_crossing'], alpha=0.3)


Past and future trajectory

In [ ]:
helper = PredictHelper(nuscenes)

sample_token = nuscenes.sample[20]['token']
anns = nuscenes.sample[20]['anns']
instance_token = nuscenes.get('sample_annotation', anns[0])['instance_token']

past = helper.get_past_for_agent(instance_token, sample_token, 2, in_agent_frame=False)
future = helper.get_future_for_agent(instance_token, sample_token, 6, in_agent_frame=False)

plt.figure(figsize=(6,6))
plt.plot(past[:,0], past[:,1], 'bo-', label="Past")
plt.plot(future[:,0], future[:,1], 'ro-', label="Future (GT)")
plt.legend()
plt.title("Trajectory of the agent (past and future)")
plt.grid()
plt.axis('equal')
plt.show()

### Input representation for trajectory prediction

The input to a model is the tensor that encodes: 

- the **static semantic map** (drivable area, walkways, lanes),
- the **past motion of nearby agents**, and
- the **target agent’s position**.

Different papers use different ways of building this tensor.  
For example:

- **CoverNet** and **MTP** rasterize the scene into a 3-channel RGB image.  
- **Rules of the Road** uses a deeper tensor with many semantic channels.

nuScenes provides a flexible module called `input_representation` that makes it
easy to construct these tensors. An input representation consists of:

1. **StaticLayerRepresentation** — how the static map is drawn  
2. **AgentRepresentation** — how surrounding agents and their history are drawn  
3. **Combinator** — how both sources are merged into a single tensor  

The MTP model uses a rasterized bird’s-eye-view image combining static layers and
agent boxes. Below we generate the exact raster input used by MTP.

In [ ]:
static_layer_rasterizer = StaticLayerRasterizer(helper)
agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1)
mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

instance_token_img, sample_token_img = 'bc38961ca0ac4b14ab90e547ba79fbb6', '7626dde27d604ac28a0240bdd54eba7a'
anns = [ann for ann in nuscenes.sample_annotation if ann['instance_token'] == instance_token_img]
img = mtp_input_representation.make_input_representation(instance_token_img, sample_token_img)

plt.imshow(img)

Visualization of rasterization that model uses

In [ ]:
static_layer_rasterizer = StaticLayerRasterizer(helper)
agent_rasterizer = AgentBoxesWithFadedHistory(helper, seconds_of_history=1)
mtp_input_representation = InputRepresentation(static_layer_rasterizer, agent_rasterizer, Rasterizer())

img = mtp_input_representation.make_input_representation(instance_token, sample_token)
plt.figure(figsize=(5,5))
plt.imshow(img)
plt.title("Raster used as input for MTP model")
plt.axis('off')
plt.show()


Data used by model

In [ ]:
vel = helper.get_velocity_for_agent(instance_token, sample_token)
acc = helper.get_acceleration_for_agent(instance_token, sample_token)
hcr = helper.get_heading_change_rate_for_agent(instance_token, sample_token)

print("Velocity:", vel)
print("Acceleration:", acc)
print("Heading Change Rate:", hcr)

In [ ]:
plt.imshow(img)
plt.title("Visual REAL input for MTP model")
plt.axis('off')
plt.show()

In [ ]:
plt.plot(future[:,0], future[:,1], 'ro-')
plt.title("Trajectory Ground Truth used to train MTP model")
plt.grid()
plt.axis('equal')
# title for x and y axes
plt.xlabel("X position (m)")
plt.ylabel("Y position (m)")
plt.show()

## Lane-Level Map Information (not used in our prediction)

The nuScenes Map API also provides detailed lane-level geometry, including lane
centerlines, connectivity, curvature, and discretized arcline paths. These
features are extremely useful for tasks such as route planning or lane
prediction.

However, the official nuScenes Prediction Challenge — and the MTP model we use
in this project — operate exclusively in a rasterized bird’s-eye-view
representation. Therefore, **lane-level map information is not used in our
training or evaluation pipeline**.

We include this small example only for educational purposes, to show the type of
high-definition map features available in the dataset but not required for our
trajectory prediction task.

In [ ]:
# To help query lane center line information
nusc_map = NuScenesMap(map_name='singapore-onenorth', dataroot=DATAROOT)

# To get the closest lane to a location. Get_lane_record to know 
# internal data representation of a lane.

# Conectivity of the lanes: get_outgoing_lanes, get_incoming_lane
x, y, yaw = 395, 1095, 0
closest_lane = nusc_map.get_closest_lane(x, y, radius=2)
closest_lane

In [ ]:
lane_record = nusc_map.get_arcline_path(closest_lane)
lane_record

In [ ]:
nusc_map.get_incoming_lane_ids(closest_lane)

In [ ]:
nusc_map.get_outgoing_lane_ids(closest_lane)

In [ ]:
poses = arcline_path_utils.discretize_lane(lane_record, resolution_meters=1)
poses

In [ ]:
# Given a query pose, find the closest pose on a lane
closest_pose_on_lane, distance_along_lane = arcline_path_utils.project_pose_to_lane((x, y, yaw), lane_record)
print(x, y, yaw)
closest_pose_on_lane

In [ ]:
distance_along_lane

In [ ]:
# Find lenght of a lane
arcline_path_utils.length_of_lane(lane_record)

In [ ]:
# Compute curvature of a lane
# 0 means it is a straight lane.
arcline_path_utils.get_curvature_at_distance_along_lane(distance_along_lane, lane_record)